网络爬取之前的贮备工作
    检查robet.txt文件
    检查sitemap文件内容
    估算网站大小（渐变方法是通过google搜索site：+网络地址的方法来粗略确认网站的大小）
    识别网站使用的技术（builtwith.parse(网络主机地址)）可以显示网站的信息,见下面的例子
    寻找网站所有者（通过whois模块）
    

In [10]:
import builtwith
import whois
website='http://example.webscraping.com'
data=builtwith.parse(website)
for i in data:
    print(data[i])
print('\n\n\n')
data=whois.whois(website)
for i in data:
    print(data[i])

['Nginx']
['Web2py', 'Twitter Bootstrap']
['Python']
['jQuery', 'Modernizr', 'jQuery UI']




WEBSCRAPING.COM
GoDaddy.com, LLC
whois.godaddy.com
None
[datetime.datetime(2013, 8, 20, 8, 8, 30), datetime.datetime(2013, 8, 20, 8, 8, 29)]
2004-06-26 18:01:19
2020-06-26 18:01:19
['NS1.WEBFACTION.COM', 'NS2.WEBFACTION.COM', 'NS3.WEBFACTION.COM', 'NS4.WEBFACTION.COM']
['clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited', 'clientRenewProhibited https://icann.org/epp#clientRenewProhibited', 'clientTransferProhibited https://icann.org/epp#clientTransferProhibited', 'clientUpdateProhibited https://icann.org/epp#clientUpdateProhibited', 'clientTransferProhibited http://www.icann.org/epp#clientTransferProhibited', 'clientUpdateProhibited http://www.icann.org/epp#clientUpdateProhibited', 'clientRenewProhibited http://www.icann.org/epp#clientRenewProhibited', 'clientDeleteProhibited http://www.icann.org/epp#clientDeleteProhibited']
abuse@godaddy.com
unsigned
None
None
None
None
Vict

编写第一个网络爬虫

学习设置网络代理

In [16]:
import urllib.request as ul
from bs4 import BeautifulSoup as bl
import json
def download(website,user_agent='wswp',num_retries=3):#增加自动重新下载的参数num_retries，默认数量为3,user_agent为设置代理
    print("Downloading:",website)
    headers={'User-agent':user_agent}
    request=ul.Request(website,headers=headers)
    try:
        html=ul.urlopen(website).read()
    except ul.URLError as e:
        print("download Error:",e.reason)
        html=None
        if hasattr(e,'code') and 500<=e.code<=600:
            return(download(website,num_retries-1))
    return (html)
website='http://www.xbiquge.la/10/10489/9683462.html'
html=bl(download(website),'lxml')
infors=html.find_all('div',id_='content')
print(infors)

Downloading: http://www.xbiquge.la/10/10489/9683462.html
[]


学习使用遍历爬虫

In [ ]:
import itertools as it
for page in it.count(1):
    url='http://example.webscraping.com/view/-%d'% page
    html=download(url)
    if html is None:
        break
    else:
        print(url)
        print('\n')

Downloading: http://example.webscraping.com/view/-1
http://example.webscraping.com/view/-1


Downloading: http://example.webscraping.com/view/-2
http://example.webscraping.com/view/-2


Downloading: http://example.webscraping.com/view/-3
http://example.webscraping.com/view/-3


Downloading: http://example.webscraping.com/view/-4
http://example.webscraping.com/view/-4


Downloading: http://example.webscraping.com/view/-5
http://example.webscraping.com/view/-5


Downloading: http://example.webscraping.com/view/-6


In [ ]:
用来爬取链接的简单爬虫

In [6]:
import re
def get_links(html):
    webpage_regex=re.compile('<a[^>]+href=["\'](.*?)["\']',re.IGNORECASE)
    return(webpage_regex.findall(html))

def link_crawler(seed_url,link_regex):
    crawl_queue=[seed_url]
    while crawl_queue:
        url=crawl_queue.pop()
        html=download(url)
        for link in get_links(html):
            if re.match(link_regex,link):
                crawl_queue.append(link)
    list=set(crawl_queue)
    print(list)

link_crawler("http://example.webscraping.com",'/(index/view)')

Downloading: http://example.webscraping.com


TypeError: expected string or bytes-like object

In [8]:
import sys
import json
import urllib.request as ul


def ip_search(ip):
    host = 'http://saip.market.alicloudapi.com'
    path = '/ip'
    method = 'GET'
    appcode = '91fa754784494761b450672aba2d3efb'
    querys = 'ip='+ip
    bodys = {}
    url = host + path + '?' + querys
    head={'Authorization':'APPCODE '+appcode}
    request = ul.Request(url,headers=head)
    response = ul.urlopen(request)
    content = response.read()
    info=json.loads(content)
    return(info)
ip=input('请输入需要查询的IP地址:')
info=ip_search(ip)
k=info['showapi_res_body']
print('IP地址对应的信息如下:'+'\n')
for i in k:
    print(i,k[i])
c=input('请按任意键结束查询。。。。。。')

请输入需要查询的IP地址:223.5.5.5
IP地址对应的信息如下:

ret_code 0
continents 亚洲
country 中国
region 浙江
city 杭州
county 
isp AliDNS
city_code 330100
en_name China
en_name_short CN
lnt 120.153576
lat 30.287459
ip 223.5.5.5
请按任意键结束查询。。。。。。
